In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_ext = data_dir/'external'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.insert(0, str(project_parent/'src'))
import dm

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=os.getenv('ESRI_GIS_PASSWORD')
)

gis

D:\projects\demographic-modeling-module\src\dm\_modify_geoaccessor.py:30: UserWarning: registration of accessor <class 'dm._modify_geoaccessor.GeoAccessorIO'> under name 'spatial' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexistingattribute with the same name.
  class GeoAccessorIO(GeoAccessor):


GIS @ https://geoai.maps.arcgis.com version:8.2

In [3]:
usa_local = dm.Country('USA')

usa_local.geographies

,name,alias,col_id,col_name,feature_class_path
0,block_groups,Block Groups,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
1,census_tracts,Census Tracts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
2,cities_and_towns_places,Cities and Towns (Places),ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
3,zip_codes,ZIP Codes,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
4,county_subdivisions,County Subdivisions,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
5,counties,Counties,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
6,cbsas,CBSAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
7,congressional_districts,Congressional Districts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
8,dmas,DMAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
9,states,States,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...


In [4]:
df_cbsa = usa_local.cbsas.get('tacoma')

df_cbsa

,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-13651055.7226, 5968866.240900002..."


In [33]:
usa_local = dm.Country('USA')
dma_bg = usa_local.dmas.get('seattle')
dma_bg

,ID,NAME,SHAPE
0,819,"Seattle-Tacoma, WA","{""rings"": [[[-13808354.615899999, 5826802.5746..."


In [35]:
dma_bg.geo_level.level(0)

TypeError: __init__() missing 1 required positional argument: 'country'

In [29]:
df_geo.iloc[:idx].name

0               block_groups
1              census_tracts
2    cities_and_towns_places
3                  zip_codes
4        county_subdivisions
5                   counties
Name: name, dtype: object

In [30]:
nm = 'block_groups'

In [31]:
self[nm] = dm.country.GeographyLevel(nm, usa_local, self.SHAPE).get()

In [32]:
self.block_groups

0    GeographyLevel: block_groups
Name: block_groups, dtype: object